In [1]:
import time
import torch, torchvision
from torch.autograd import Variable
import numpy as np
import pandas as pd
import os
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import datetime

from DBFNet.Model import *
import warnings

In [2]:
def compute_distance_km(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = lat1.astype('float'), lon1.astype('float'), lat2.astype('float'), lon2.astype('float')
    # 批量计算地球上两点间的球面距离
    R = 6371e3  # 地球半径（米）
    phi_1, phi_2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.power(np.sin(delta_phi / 2), 2) + np.cos(phi_1) * np.cos(phi_2) * np.power(np.sin(delta_lambda / 2), 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c / 1000.

In [3]:
def data_scaler(x, feature_range=[0, 1]):
    scaler = MinMaxScaler(feature_range=feature_range)
    shape = x.shape
    x = x.reshape(shape[0], -1)
    x = scaler.fit_transform(x)
    x = x.reshape(shape)
    return x, scaler

In [4]:
class TrainLoader(Dataset):
    def __init__(self, x, y, g, standard):
        self.x = x
        self.y = y
        self.g = g
        self.s = standard

    def __getitem__(self, item):
        return self.x[item], self.g[item, :4], self.y[item], self.g[item, 4:], self.s[item]

    def __len__(self):
        return len(self.x)

In [5]:
def test_2018_2021(cfg1=TC_Encoder_config, cfg2=Map_Encoder_config, checkpoint_folder='./model/',
                   xtc_path='./CMA_dataset/xtc.npy', ytc_path='./CMA_dataset/ytc.npy', gph_path='./gph.npy',
                   batch_size=32, is_save=True, checkpoint='./model/model_demo.pth'):
    warnings.filterwarnings("ignore")  # 关闭warning

    # 载入完整数据集
    print('载入完整数据集...')
    xtc = np.load(xtc_path, allow_pickle=True).astype(float)
    ytc = np.load(ytc_path, allow_pickle=True).astype(float)
    gph = np.load(gph_path, allow_pickle=True).astype(float)
    test_index = [*range(22227, 24614)] # 2018年至2021年的台风数据索引
    standard = xtc[:, -1, :2].reshape(-1, 1, 2)

    # 归一化
    xtc, _ = data_scaler(xtc)
    gph, _ = data_scaler(gph)

    # 打包数据集
    test_dataset = DataLoader(TrainLoader(xtc[test_index], ytc[test_index], gph[test_index], standard[test_index]),
                              batch_size=batch_size, shuffle=False)
    del xtc, ytc, standard, gph

    model = DBFNet(cfg1, cfg2)
    if torch.cuda.is_available():
        model = model.cuda()
    state_dict = torch.load(checkpoint)
    model.load_state_dict(state_dict)

    # 在测试集上预测
    print('测试集上预测：')
    model.eval()
    pred_save, truth_save, standard_save = [], [], []
    for X1, X2, Y1, _, S in tqdm(test_dataset):
        if torch.cuda.is_available():
            x1 = X1.float().cuda()
            x2 = X2.float().cuda()
            y1 = Y1.float().cuda()
            s = S.float().cuda()
        else:
            x1 = X1.float()
            x2 = X2.float()
            y1 = Y1.float()
            s = S.float()
        pred, _ = model(x1, x2)
        pred_save.append(pred.detach().cpu().numpy())
        truth_save.append(y1.detach().cpu().numpy())
        standard_save.append(s.detach().cpu().numpy())
    # 做反归一化
    pred_save = np.concatenate(pred_save, axis=0)
    truth_save = np.concatenate(truth_save, axis=0)
    standard_save = np.concatenate(standard_save, axis=0)
    # pred_save = data_scaler_inverse(pred_save, y_scaler)
    # 计算最终预测结果
    pred = pred_save + standard_save
    truth = truth_save + standard_save
    for i in range(1, 5):
        pred_lat, pred_lon, truth_lat, truth_lon = pred[:, i - 1, 0], pred[:, i - 1, 1], truth[:, i - 1, 0], truth[:,
                                                                                                             i - 1, 1]
        distance = compute_distance_km(pred_lat, pred_lon, truth_lat, truth_lon)
        print(str(i * 6) + 'h APE', distance.mean(), ' MAE latitude:', np.abs(pred[:, i - 1, 0] - truth[:, i - 1, 0]).mean(), 
              ' MAE longitude:', np.abs(pred[:, i - 1, 1] - truth[:, i - 1, 1]).mean(), ' RMSE latitude:', np.sqrt(((pred[:, i - 1, 0] - truth[:, i - 1, 0]) ** 2).mean()),
             ' RMSE longitude:', np.sqrt(((pred[:, i - 1, 1] - truth[:, i - 1, 1]) ** 2).mean()))

In [6]:
test_2018_2021()

载入完整数据集...
测试集上预测：


100%|██████████| 75/75 [00:01<00:00, 58.79it/s]

6h APE 39.73501210493949  MAE latitude: 0.2209794  MAE longitude: 0.27022302  RMSE latitude: 0.30970362  RMSE longitude: 0.42509833
12h APE 83.73567743841966  MAE latitude: 0.4678313  MAE longitude: 0.57108516  RMSE latitude: 0.64640456  RMSE longitude: 0.85537344
18h APE 133.86184621412266  MAE latitude: 0.75017697  MAE longitude: 0.92322457  RMSE latitude: 1.027398  RMSE longitude: 1.343041
24h APE 190.3670649653396  MAE latitude: 1.0742958  MAE longitude: 1.3113072  RMSE latitude: 1.4749578  RMSE longitude: 1.9104049
